# _Physarum_ Learner

This notebook presents the implementation and the documentation of the "Physarum Learner" algorithm, as proposed in __[Physarum Learner: A bio-inspired way of learning structure from data](dx.doi.org/10.1016/j.eswa.2014.03.002)__, by Schön et al.

## Import section

Here we present the libraries used during the execution of the code.

The purpose of each library is:
-  _pandas_ is needed in order to import the data from the chosen dataset in an easy way;
-  _numpy_ is used for several math functionalities used throughout the code;
-  _networkx_ has a lot of graph structures and utilities needed for the implementation of a Bayesian Network based code;
-  _pgmpy_ has several functionalities related to Bayesian Networks, and is invoked here mostly for its BDeu scoring functions;
-  _time_ is needed to get the time spent on the execution of the main algorithm.

In [8]:
import pandas as pd
import numpy as np
import networkx as nx
from pgmpy.estimators import BdeuScore
import time
from scipy.linalg import lapack


## Custom Class

This code section presents the class "info", used to store edges information, according to the original _Physarum Solver_ algorithm. The _cond_ variable corresponds to that path's conductance $ D_{ij} $, and the _length_ variable stores the path's length $ L_{ij} $. For more information, please check __[_Physarum_ solver: A biologically inspired method of road-network navigation](https://doi.org/10.1016/j.physa.2006.01.053)__, by Tero, A., Kobayashi, R., and Nakagaki, T.

In [9]:
class info:
    def __init__(self, cond, length):
        self.cond = cond
        self.length = length
        

## Global variables

In order to create the least amount of overhead when calling several nested functions, and to access the needed information without creating a copy of the whole structure, these variables were declared and used as global, which means that every function is able to see and change their contents as needed. A brief explanation about the usage of each one follows:
-  _maze_: stores the _Physarum_ maze which will be iterated upon, somewhat according to the aforementioned articles;
-  _partNetwork_: contais the best network found so far on a particular ensemble;
-  _bestNetwork_: is used to store the best network found among all ensembles;
-  _bdeu_: object of _BdeuScore_ class from _pgmpy_, used for the scoring of the networks found;
-  _df_: the dataframe which contains the data to be used on the creation of the networks;
-  _Dt_: base conductance used to several calculations.

In [10]:
maze = nx.Graph()
partNetwork = nx.DiGraph()
bestNetwork = nx.DiGraph()
bdeu = 0
df = 0
Dt = 0.8


## Auxiliary functions

Because of the high complexity of the algorithm implemented, several auxiliary functions had to be created in order to perform repetitive tasks and allow for a better code in terms of legibility. This section presents each one of them, as well as some comments about its usage.

### Partial _Physarum_ solver

As the core of the algorithm implemented, the so-called _pPhySolve_ function here presented is used to perform one single iteration of the original _Physarum_ solver (that's why we called it "partial" here). As arguments, the function receives two numbers that identify two nodes in the global _Physarum_ maze already introduced, and updates the conductance of the edges accordingly.

In [11]:
def pPhySolve(source, sink):    
    
    global maze
    
    #Informations used during the method.
    Io = 3
    w = 0.5
    l = 0.2
    
    num_nodes = 0
    
    for i in maze.nodes():
        num_nodes += 1
    
    A = np.empty([num_nodes, num_nodes])
    b = np.zeros(num_nodes)
        
    #Setting up the linear system Ax=b.
    for i in range(num_nodes):
        acc = 0.0
        for j in range(num_nodes):
            if(i != j):
                if(maze.has_edge(i, j)):
                    A[i][j] = (maze.edges[i, j]['path'].cond)/(maze.edges[i, j]['path'].length)
                    acc += A[i][j]
                else:
                    A[i][j] = 0
        A[i][i] = -acc
        if (i == source):
            b[i] = -Io
        elif (i == sink):
            b[i] = Io

    #Solve the system to get partial pressures at nodes.
    pres = psolve(A, b)

    #Leveling pressures according to sink's.
    if(pres[sink] < 0):
        for i in range(num_nodes):
            if (i != sink):
                pres[i] -= pres[sink]
        pres[sink] -= pres[sink]
    else:
        for i in range(num_nodes):
            if (i != sink):
                pres[i] += pres[sink]
        pres[sink] += pres[sink]


    #Calculate the flow through each edge and update its conductance.
    for (i, j) in maze.edges():
        q_part = (maze.edges[i, j]['path'].cond)/(maze.edges[i, j]['path'].length)
        q = q_part*(pres[i]-pres[j])

        f = feedback(np.absolute(q))

        maze.edges[i, j]['path'].cond = (w*f)+(1-l*w)*maze.edges[i, j]['path'].cond
        

### Feedback function


In accordance to the original _Physarum_ solver method, the conductance of the edges is modeled by a positive feedback relation with the flux passing through that edge in a given iteration. This function implements this feedback factor, and is calculated separately in order to be easily changed for another, this one being from the same or from a different type than the one used here. Again, for more information about the feedback function types, please check the original _Physarum_ solver article.

This function implements the type-I feedback function $ f(Q_{ij}) = Q_{ij}^\mu $, where $ \mu $ is set to $1$.

In [12]:
def feedback(q):
    mi = 1
    return np.power(q, mi)


### Linear system solving

The _Physarum_ solver method relies on the creation and solution of a linear system in the form $A\cdot x = b$, where A may be singular, i. e. its determinant is zero, which prevents the regular _numpy_'s linear system solver to be directly used. As such, we had to find an implementation of a "partial solver" which uses the Moore-Penrose pseudo-inverse of a matrix. The code below was based on the one presented __[here](https://github.com/numpy/numpy/blob/master/numpy/linalg/linalg.py)__.

In [13]:
def pinv(a, rcond=1e-15):
    
    rcond = np.asarray(rcond)
    u, s, vt = np.linalg.svd(a, full_matrices=False)

    # discard small singular values
    cutoff = rcond[..., np.newaxis] * np.amax(s, axis=-1, keepdims=True)
    large = s > cutoff
    s = np.divide(1, s, where=large, out=s)
    s[~large] = 0

    res = np.matmul(np.swapaxes(vt, -1, -2),
                    np.multiply(s[..., np.newaxis], np.swapaxes(u, -1, -2)))
    return res


def psolve(a, b):
    
    lu, piv, x, info = lapack.dgesv(a, b)
    
    if (info != 0):
        newaxis = (a.ndim == b.ndim + 1)
        if newaxis:
            b = b[..., np.newaxis]
        ai = pinv(a)
        x = np.matmul(ai, b)
        if newaxis:
            x = x[..., 0]
    
    return x


### Scoring functions

The _Physarum_ learner algorithm needs a set of scoring functions in order to create and compare networks. These two functions use the Bayesian Dirichlet equivalent uniform (BDeu) score.

The first function calculates the score of a whole given network, through the sum of the scores obtained by each of its nodes and the parents from those. The second function is used to find the score improvement from creating the edge, compared to the graph not having it.

In [14]:
def calcScore(network):
    
    global df
    
    if network is None:
        return np.finfo(float).min
    
    else:
        score = 0
    
        for i in network.nodes():
            score += bdeu.local_score(df.columns[i], [df.columns[j] for j in network.predecessors(i)])
        
        return score

def calcLocalScore(network, i, j):
    
    global df
    
    auxlist = [df.columns[k] for k in network.predecessors(j)]
    
    if network.has_edge(i, j) is False:
        score1 = bdeu.local_score(df.columns[j], auxlist)
    
        score2 = bdeu.local_score(df.columns[j], auxlist+[df.columns[i]])
        
        return (score2-score1)
    
    else:
        score1 = bdeu.local_score(df.columns[j], auxlist)
        
        index = auxlist.index(df.columns[i])
        
        del auxlist[index]
        
        score2 = bdeu.local_score(df.columns[j], auxlist)
        
        return (score1-score2)


### Checking for cycles

According to the theory of Bayesian networks, the graph representing it has to have no edges. To comply with this, this function simply creates an edge between two nodes whose identifiers were obtained through its parameters, and check if this operation generated a cycle on the graph. Independently of the result of the check, the edge is removed before the return of the function, to maintain the integrity of the graph. The return values are _True_ if the addition of the edge does not create a cycle, and _False_ otherwise.

In [15]:
def tryEdge(B, i, j):
    
    B.add_edge(i, j)
    
    try:
        nx.find_cycle(B, source=j, orientation='reverse')
        
    except nx.NetworkXNoCycle:
        B.remove_edge(i, j)
        return True
        
    else:
        B.remove_edge(i, j)
        return False
        

## Core functions

With the "minor" functions presented, we now introduce the three core functions this algorithm is based upon. For more information on any code fragment, please read the comments on the source code provided and/or access the _Physarum_ learner original article.

### Evaluate function

The purpose of this function, in general terms, is to evaluate the _Physarum_ maze and decide which edges will be put on the network _partNetwork_, in order to decrease its BDeu score. To achieve that it checks the quality of the edges between the nodes, the possibility of adding the edge according to the "no cycle" restriction, and the improvement caused by this addition to the overall network structure.


In [16]:
def evaluate():
    
    global maze
    global partNetwork, bestNetwork
    
    whitelist = []
    
    #Checks all the edges on the maze.
    for (i, j) in maze.edges():
        
        #If the edge has a good conductance according to the solver's algorithm
        #    it is added to the whitelist list.
        if(maze.edges[i, j]['path'].cond > Dt):
            whitelist.append((i, j))
        
        #If it hasn't, the edge on the partNetwork is removed (if it exists).
        else:
            if(partNetwork.has_edge(i, j)):
                partNetwork.remove_edge(i, j)
            
            elif(partNetwork.has_edge(j, i)):
                partNetwork.remove_edge(j, i)
    
    #For every edge on the list:
    while whitelist:
        scoreImp = 0
        bestCon = (0, 0)
        
        for (i, j) in whitelist:
            
            #Remove the edge on partNetwork, to guarantee the best direction for it.
            if(partNetwork.has_edge(i, j)):
                partNetwork.remove_edge(i, j)
            
            elif(partNetwork.has_edge(j, i)):
                partNetwork.remove_edge(j, i)
            
            #If the edge from "i" to "j" doesn't add a cycle to the graph, 
            if(tryEdge(partNetwork, i, j) is True):
                
                #Calculate the score improvement of the addition.
                score_ij = calcLocalScore(partNetwork, i, j)
                
            #Otherwise, if the addition would create a cycle, gives to the improvement
            #    the worst score possible.
            else:
                score_ij = np.finfo(float).min
            
            #The same from the last couple of lines, checking the other direction of the edge.
            if(tryEdge(partNetwork, j, i) is True):
                score_ji = calcLocalScore(partNetwork, j, i)
                
            else:
                score_ji = np.finfo(float).min
                               
            #Pick whichever direction would result in the best improvement on the network.
            change = max(score_ij, score_ji)
            
            #Stores the best connection found so far.
            if(change > scoreImp):
                scoreImp = change
                
                if (score_ij >= score_ji):
                    bestCon = (i, j)
                    
                else:
                    bestCon = (j, i)
        
        #iIf no connections that improve the score were found.
        if (bestCon == (0, 0)):
           
            #Gives feedback to the edge on the Physarum maze
            for (i, j) in whitelist:
                giveFeedback(i, j)
            
            #If the partNetwork's score is better than the found so far, overwrite it.
            if (calcScore(partNetwork) > calcScore(bestNetwork)):
                bestNetwork = partNetwork.copy()
                
            return
        
        #If there's a connection that improves the score
        (i, j) = bestCon
        
        #Put it on the partNetwork
        partNetwork.add_edge(i, j)
        
        #Remove it from the whitelist, and give feedback to the edge.
        try:
            whitelist.index((i, j))
        
        except ValueError:
            del whitelist[whitelist.index((j, i))]
            giveFeedback(j, i)
        
        else:
            del whitelist[whitelist.index((i, j))]
            giveFeedback(i, j)
    
    #If the partNetwork's score is better than the found so far, overwrite it.
    if (calcScore(partNetwork) > calcScore(bestNetwork)):
        bestNetwork = partNetwork.copy()
        
    return
        


### Update of the conductance

In order to preserve the best connections found so far according to the score of the Bayesian network on the original maze, and to inhibit the worst ones to be chosen in the next iterations of the algorithm, a bias is introduced on the conductance calculation for each edge of the maze.


In [17]:
def giveFeedback(i, j):
    
    global maze
    global partNetwork
    global df
    
    #Local parameter
    k = 3.0
    
    #Find which is the best direction to be added to the network.
    score_ij = calcLocalScore(partNetwork, i, j)
    
    score_ji = calcLocalScore(partNetwork, j, i)
    
    #If the direction chosen is from "i" to "j"
    if(score_ij >= score_ji):
        
        #Get "j" parents on the network.
        auxlist = [df.columns[k] for k in partNetwork.predecessors(j)]
        
        if partNetwork.has_edge(i, j) is True:
            
            B = bdeu.local_score(df.columns[j], auxlist)
            
            index = auxlist.index(df.columns[i])
            
            del auxlist[index]
            
        #Calculate B.
            B = B / bdeu.local_score(df.columns[j], auxlist)
            
        else:
            
            B = bdeu.local_score(df.columns[j], auxlist+[df.columns[i]]) / bdeu.local_score(df.columns[j], auxlist)
    
    #If the direction chosen is from "j" to "i"
    else:
        
        #Get "i" parents on the network.
        auxlist = [df.columns[k] for k in partNetwork.predecessors(i)]
        
        if partNetwork.has_edge(j, i) is True:
            
            B = bdeu.local_score(df.columns[i], auxlist)
            
            index = auxlist.index(df.columns[j])
            
            del auxlist[index]
            
        #Calculate B.
            B = B / bdeu.local_score(df.columns[i], auxlist)
            
        else:
            
            B = bdeu.local_score(df.columns[i], auxlist+[df.columns[j]]) / bdeu.local_score(df.columns[i], auxlist)
        
    #Calculate the biased conductance
    aux = maze.edges[i, j]['path'].cond + k*(1-B)
    
    #If the value is above the threshold of 2.5, it becomes 2.5.
    if(aux >= 2.5):
        maze.edges[i, j]['path'].cond = 2.5
    
    #If not, it becomes the biased value.
    else:
        maze.edges[i, j]['path'].cond = aux


### Main function

This is the main function of the program, from where it starts and where it finishes. The global structures are first set up here, as well as the parameters controlling the number of ensembles found and iterations for each of those. Important of note is that, after an ensemble is finished and before a new one is started, the maze has its conductances reset, and the _partNetwork_ is randomly reconnected.

In [18]:
def main():
    
    global maze
    global bestNetwork, partNetwork
    global bdeu, df
    
    #Number of ensembles created.
    Ensemble = 1
    
    #Number of iterations for the construction of every ensemble.
    MaxIt = 5
    
    #Initial setup of useful structures.
    bestNetwork = None
    
    nodelist = []
    edgeslist = []
    
    #Acquisition of data.
    df = pd.read_csv("data_asia.csv")
    
    bdeu = BdeuScore(df, equivalent_sample_size=5)
    
    #More setup.
    num_nodes = len(df.columns)
    
    for i in range(num_nodes):
        maze.add_node(i)
        partNetwork.add_node(i, label=list(df.columns)[i])
    
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            maze.add_edge(i, j, path=info(np.random.uniform(0.78, 0.79), 1))
            
            if(np.random.uniform(0, 1) < 0.5):
                partNetwork.add_edge(i, j)
                
            else:
                partNetwork.add_edge (j, i)
    
    #Start timer.
    start = time.time()
    
    #*********Main loop.*********
    
    #For each ensemble to be built.
    for e in range(Ensemble):
        
        #For each iteration.
        for it in range(MaxIt):
            
            #Create list with every pair of nodes on the maze.
            for i in range(num_nodes):
                for j in range(i+1, num_nodes):
                    nodelist.append((i, j))
            
            #While there are still pairs on the list.
            while nodelist:
                
                #Pick a random pair and remove it from the list.
                index = np.random.randint(0, len(nodelist))
                
                source, sink = nodelist.pop(index)
                
                #Perform one iteration of the Phsarum solver.
                pPhySolve(source, sink)
                
                #If the conductance of the edge for this pair is below the threshold,
                #    gives it a value above the threshold, in order to give it a chance
                #    of entering the network.
                if(maze.edges[source, sink]['path'].cond < Dt):
                    maze.edges[source, sink]['path'].cond = Dt + 0.01
                
                #Evaluate the maze according to the conductances.
                evaluate()
                
                #If the partNetwork's score is better than the found so far, overwrite it.
                if (calcScore(partNetwork) > calcScore(bestNetwork)):
                    bestNetwork = partNetwork.copy()
        
        #End of he iterations, the edges on the partNetwork are removed.
        for (i, j) in partNetwork.edges():
            edgeslist.append((i, j))
            
        for (i, j) in edgeslist:
            partNetwork.remove_edge(i, j)
            
        del edgeslist[:]
        
        #The maze's conductances are reset, and the edges on partNetwork are randomly
        #    Recreated.
        for (i, j) in maze.edges():
            maze.edges[i, j]['path'].cond = np.random.uniform(0.78, 0.79)
            
            if(np.random.uniform(0, 1) < 0.5):
                partNetwork.add_edge(i, j)
                
            else:
                partNetwork.add_edge (j, i)
            
    #End of the main loop.
    
    #Get the time elapsed
    elapsed = time.time() - start
       
    #Print the edges of the best network found.
    for (i, j) in bestNetwork.edges():
        print(df.columns[i], "->", df.columns[j])
    
    #Print the score of the best network.
    print("Final score -> ", calcScore(bestNetwork))
    
    #Print the time required for the main loop to complete.
    print("Time elapsed -> ", elapsed)
    
if __name__ == "__main__":
    main()


A -> E
S -> B
T -> E
T -> A
T -> B
T -> X
L -> E
L -> S
L -> X
B -> D
E -> X
E -> D
Final score ->  -11109.241257785368
Time elapsed ->  77.62594223022461
